In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 33924665
paper_name = 'jin_liu_2021' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [8]:
original_data = pd.read_excel('raw_data/original data.xlsx', sheet_name='Sheet1', skiprows=1)

In [9]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 4129 x 30


In [10]:
original_data.head()

,Array ORF,Array Name,Normalized colony size (EXPERIMENT),Normalized colony std. dev. (EXPERIMENT),Normalized colony size (CONTROL),Normalized colony std. dev. (CONTROL),Score,Score stdev,p-Value,Additional information,...,Unnamed: 20,Unnamed: 21,Normalized colony size (EXPERIMENT).2,Normalized colony std. dev. (EXPERIMENT).2,Normalized colony size (CONTROL).2,Normalized colony std. dev. (CONTROL).2,Score.2,Score stdev.2,p-Value.2,Additional information.2
0,YGL007W,YGL007W,0.11528,0.012,0.83497,0.012,-0.72188,0.012,0.00031,NaN,...,NaN,NaN,0.25703,0.015,0.86201,0.064,-0.62125,0.015,0.00005,NaN
1,YOR033C,EXO1,0.10582,0.019,0.79420,0.021,-0.68147,0.019,0.00162,NaN,...,NaN,NaN,0.46136,0.011,0.73788,0.017,-0.27375,0.011,0.00000,NaN
2,YAL002W,VPS8,0.44421,0.020,1.15528,0.101,-0.67837,0.020,0.00002,NaN,...,NaN,NaN,0.63835,0.010,1.11008,0.013,-0.46565,0.010,0.00000,NaN
3,YOL095C,HMI1,0.11675,0.012,0.79231,0.028,-0.67351,0.012,0.00339,status=JK,...,NaN,NaN,0.44620,0.003,0.68030,0.014,-0.23543,0.003,0.00002,status=JK
4,YNL081C,SWS2,0.04806,0.036,0.72179,0.004,-0.67178,0.036,0.07503,NaN,...,NaN,NaN,0.24750,0.036,0.75068,0.044,-0.49466,0.036,0.00080,NaN


In [11]:
original_data['orf'] = original_data['Array ORF'].astype(str)

In [12]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [13]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [14]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [Array ORF, Array Name, Normalized colony size (EXPERIMENT), Normalized colony std. dev. (EXPERIMENT), Normalized colony size (CONTROL), Normalized colony std. dev. (CONTROL), Score, Score stdev, p-Value, Additional information, Unnamed: 10, Unnamed: 11, Normalized colony size (EXPERIMENT).1, Normalized colony std. dev. (EXPERIMENT).1, Normalized colony size (CONTROL).1, Normalized colony std. dev. (CONTROL).1, Score.1, Score stdev.1, p-Value.1, Additional information.1, Unnamed: 20, Unnamed: 21, Normalized colony size (EXPERIMENT).2, Normalized colony std. dev. (EXPERIMENT).2, Normalized colony size (CONTROL).2, Normalized colony std. dev. (CONTROL).2, Score.2, Score stdev.2, p-Value.2, Additional information.2, orf]
Index: []

[0 rows x 31 columns]


In [15]:
data_cols = ['Score','Score.1','Score.2']
for d in data_cols:
    original_data[d] = pd.to_numeric(original_data[d])

In [16]:
original_data['data'] = original_data[data_cols].mean(axis=1)

In [17]:
original_data.set_index('orf', inplace=True)

In [18]:
original_data = original_data[['data']].copy()

In [19]:
original_data = original_data.groupby(original_data.index).mean()

In [20]:
original_data.shape

(4081, 1)

# Prepare the final dataset

In [21]:
data = original_data.copy()

In [22]:
dataset_ids = [21905]
datasets = datasets.reindex(index=dataset_ids)

In [23]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [24]:
data.head()

dataset_id,21905
data_type,value
orf,
YAL002W,-0.595443
YAL004W,-0.145930
YAL005C,-0.078347
YAL007C,0.041613
YAL008W,0.020573


## Subset to the genes currently in SGD

In [25]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 20


In [26]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,21905
,data_type,value
gene_id,orf,
2,YAL002W,-0.595443
1863,YAL004W,-0.145930
4,YAL005C,-0.078347
5,YAL007C,0.041613
6,YAL008W,0.020573


# Normalize

In [27]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [28]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [29]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id          21905          
data_type           value    valuez
gene_id orf                        
2       YAL002W -0.595443 -6.006112
1863    YAL004W -0.145930 -1.519344
4       YAL005C -0.078347 -0.844769
5       YAL007C  0.041613  0.352599
6       YAL008W  0.020573  0.142590

# Print out

In [30]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [31]:
from IO.save_data_to_db3 import *

In [32]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 33924665...
Inserting the new data...


100%|██████████| 1/1 [00:05<00:00,  5.80s/it]

Updating the data_modified_on field...
